<a href="https://colab.research.google.com/github/ujkim25/sparta_ai_engineering/blob/main/RangChain_output_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain==0.3.27
!pip install langchain_community==0.3.31
!pip install langchain_core==0.3.79
!pip install langchain_experimental==0.3.4
!pip install langchain_huggingface==0.3.0
!pip install langchain_openai==0.3.28
!pip install langchain_text_splitters==0.3.11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 6.9 MB/s eta 0:00:00


In [2]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field # json 형태의 output 강제

In [4]:
model = ChatOpenAI(model_name = "gpt-4.1", temperature=0)

In [5]:
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [6]:
question = "지구 온난화의 심각성에 대해 알려주세요"

In [7]:
parser = JsonOutputParser(pydantic_object=Topic)

In [8]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "주제에 대한 간결한 설명", "title": "Description", "type": "string"}, "hashtags": {"description": "해시태그 형식의 키워드(2개 이상)", "title": "Hashtags", "type": "string"}}, "required": ["description", "hashtags"]}
```


In [12]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system","당신은 친절한 AI 어시스턴트입니다. 질문에 간결하게 답변하세요"),
        ("user","#Format: {format_instructions}\n\n#Question: {question}")
    ]
)

In [13]:
prompt = prompt_template.partial(
    format_instructions=parser.get_format_instructions()
)

In [14]:
chain = prompt | model | parser

In [15]:
answer = chain.invoke(
    {
        "question":question
    }
)

In [24]:
type(answer)

dict

In [16]:
answer

{'description': '지구 온난화는 지구 평균 기온이 상승하면서 기후 변화, 해수면 상승, 생태계 파괴 등 심각한 문제를 초래하고 있습니다. 이는 인류와 자연 모두에게 큰 위협이 됩니다.',
 'hashtags': '#지구온난화 #기후변화 #환경위기 #생태계파괴'}

In [17]:
question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 'description'에 관련 키워드는 'hashtags'에 담아주세요"

parser = JsonOutputParser()

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system","당신은 친절한 AI 어시스턴트입니다. 질문에 간결하게 답변하세요"),
        ("user","#Format: {format_instructions}\n\n#Question: {question}")
    ]
)

prompt = prompt_template.partial(
    format_instructions=parser.get_format_instructions()
)

In [18]:
prompt
#partial을 통해 format_instructions을 제공했기에 input_variables에 question만 남아있다

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={'format_instructions': 'Return a JSON object.'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 친절한 AI 어시스턴트입니다. 질문에 간결하게 답변하세요'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'question'], input_types={}, partial_variables={}, template='#Format: {format_instructions}\n\n#Question: {question}'), additional_kwargs={})])

In [19]:
prompt_template

ChatPromptTemplate(input_variables=['format_instructions', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 친절한 AI 어시스턴트입니다. 질문에 간결하게 답변하세요'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'question'], input_types={}, partial_variables={}, template='#Format: {format_instructions}\n\n#Question: {question}'), additional_kwargs={})])

In [20]:
chain = prompt | model | parser

In [21]:
response = chain.invoke(
    {
        "question":question
    }
)

In [22]:
response

{'description': '지구 온난화는 대기 중 온실가스 농도가 증가하면서 지구 평균 기온이 점차 상승하는 현상입니다. 이는 주로 이산화탄소, 메탄 등 온실가스의 배출이 원인으로, 기후 변화, 해수면 상승, 극한 기상 현상 등 다양한 환경 문제를 일으킵니다.',
 'hashtags': ['#지구온난화', '#온실가스', '#기후변화', '#이산화탄소', '#해수면상승', '#환경문제']}

In [23]:
type(response)

dict

In [25]:
email_conversation = """From: 김철수 (chulsoo.kim@sparta.me)
To: 이은채 (eunchae@sparta.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [26]:
from langchain_core.prompts import PromptTemplate

In [27]:
prompt = PromptTemplate.from_template(
    "다음의 이메일 내용 중 중요한 내용을 추출해주세요. \n\n {email_conversation}"
)

llm = ChatOpenAI(model_name = "gpt-4.1", temperature=0)

chain = prompt | llm

In [28]:
answer = chain.stream(
    {"email_conversation":email_conversation}
)

for chunk in answer:
    print(chunk.content,end="",flush=True)

**중요 내용 추출:**

1. **브로슈어 요청**  
   - ZENESIS 자전거의 상세 브로슈어 요청  
   - 특히 기술 사양, 배터리 성능, 디자인 정보 필요

2. **미팅 제안**  
   - 협력 논의를 위한 미팅 제안  
   - 일정: 다음 주 화요일(1월 15일) 오전 10시  
   - 장소: 귀사(이은채 대리님 회사) 사무실

3. **목적**  
   - ZENESIS 자전거 유통 협력 및 마케팅 전략 논의

4. **발신자 정보**  
   - 김철수 상무이사, 바이크코퍼레이션

In [29]:
class EmailSummary(BaseModel):
    person: str = Field(description = "메일을 보낸 사람")
    email: str = Field(description = "메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description = "메일 제목")
    summary: str = Field(description = "메일 본문을 요약한 텍스트")
    date: str = Field(description = "메일 본문에 언급된 미팅 날짜와 시간")

from langchain_core.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [30]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [31]:
prompt = PromptTemplate.from_template(
    """
    당신은 친절한 AI 어시스턴트입니다. 다음 질문과 내용에 대해 한국어로 대답해주세요

    Question:{question}

    Email Conversation:
    {email_conversation}

    Format:
    {format}
    """
)

In [32]:
prompt = prompt.partial(format = parser.get_format_instructions())

In [33]:
chain = prompt | llm

In [34]:
response = chain.stream(
    {
        "question":question,
        "email_conversation":email_conversation
    }
)

output=""
for chunk in response:
    output += chunk.content

In [35]:
print(output)

{
  "person": "김철수",
  "email": "chulsoo.kim@sparta.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "김철수 상무가 이은채 대리에게 ZENESIS 자전거의 브로슈어(특히 기술 사양, 배터리 성능, 디자인 정보)를 요청하고, 유통 및 마케팅 협력 논의를 위해 1월 15일(화) 오전 10시에 미팅을 제안하는 내용입니다.",
  "date": "1월 15일(화) 오전 10시"
}


In [36]:
type(output)

str

In [37]:
structured_output = parser.parse(output)

In [38]:
structured_output

EmailSummary(person='김철수', email='chulsoo.kim@sparta.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='김철수 상무가 이은채 대리에게 ZENESIS 자전거의 브로슈어(특히 기술 사양, 배터리 성능, 디자인 정보)를 요청하고, 유통 및 마케팅 협력 논의를 위해 1월 15일(화) 오전 10시에 미팅을 제안하는 내용입니다.', date='1월 15일(화) 오전 10시')

In [39]:
from langchain_core.documents import Document

In [41]:
document = Document(page_content = "안녕하세요? 랭체인 도큐먼트입니다")

In [42]:
document

Document(metadata={}, page_content='안녕하세요? 랭체인 도큐먼트입니다')

In [43]:
document.metadata["source"] = "Sparta"
document.metadata["page"] = 1
document.metadata["author"] = "Sparta"

document

Document(metadata={'source': 'Sparta', 'page': 1, 'author': 'Sparta'}, page_content='안녕하세요? 랭체인 도큐먼트입니다')

In [44]:
!pip install pypdf rapidocr-onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.3 MB/s eta 0:00:00


In [45]:
import os

os.listdir()

['.config',
 'word_sample.docx',
 '디지털 정부혁신 추진계획.hwp',
 'appendix-keywords-EUCKR.txt',
 'reference.txt',
 'appendix-keywords-CP949.txt',
 'titanic.csv',
 'audio_utils.py',
 'appendix-keywords-utf8.txt',
 '지식비타민_돈 되는 잠, 슬리포노믹스.pdf',
 'appendix-keywords.txt',
 'chain-of-density.txt',
 'titanic.xlsx',
 'client.html',
 'sample-ppt.pptx',
 'people.json',
 'sample_data']

In [49]:
FILE_PATH = "지식비타민_돈 되는 잠, 슬리포노믹스.pdf"

In [50]:
from langchain_community.document_loaders import PyPDFLoader

In [51]:
loader = PyPDFLoader(FILE_PATH)

In [52]:
docs = loader.load()

In [53]:
docs[0]

Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2024-04-01T15:50:27+09:00', 'author': '경영연구소', 'moddate': '2024-04-01T15:50:27+09:00', 'source': '지식비타민_돈 되는 잠, 슬리포노믹스.pdf', 'total_pages': 11, 'page': 0, 'page_label': '1'}, page_content='2024. 4 \n \n \n \n \n \n \n \n \n돈 되는 잠, 슬리포노믹스 \n \n \n \n \n \n \n \n \n \n \n \n \n \n  \n□ 숙면하지 못하는 한국인과 슬리포노믹스의 등장 \n□ 다양한 분야로 확산되는 슬리포노믹스 \n□ 미래 먹거리로 부상하는 슬리포노믹스 \n□ 향후 전망과 시사점')

In [54]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [55]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 0
)

In [56]:
loader = PyPDFLoader(FILE_PATH)

In [57]:
split_docs = loader.load_and_split(
    text_splitter=text_splitter
)

In [58]:
print(len(split_docs))

67


In [59]:
loader = PyPDFLoader(FILE_PATH,extract_images=True)

In [60]:
docs=loader.load()

In [61]:
print(docs[2].page_content)

2 
 숙면하지 못하는 한국인과 슬리포노믹스(Sleeponomics)의 등장 
○ 한국인의 심각한 수면 시간 부족과 낮은 수면의 질은 개인의 문제를 넘어 사회경제적 손실로 
확대됨 
• 한국은 대표적 잠 부족 국가로, 하루 평균 수면 시간은 OECD 회원국 중 최하위 수준 
- 경제협력개발기구(OECD) 2021년 조사에 따르면 한국인의 하루 평균 수면 시간은 7시간 
51분으로 일본과 더불어 최하위 수준이며, OECD 회원국 평균인 8시간 27분에 비해 30
분 이상 부족 
• 수면의 질 역시 낮으며 많은 한국인이 만족스러운 수면을 취하지 못하고 있음 
- 불면증·기면증·수면무호흡증 등의 수면 장애, 걱정, 스트레스, 취침 전 스마트폰 사용 등
의 생활 습관이 수면의 질을 떨어뜨림 
- 필립스가 2021년 3월 19일 세계 수면의 날을 맞아 전 세계 13개국1 1만 3천 명을 대
상으로 진행한 수면 조사에서 세계인의 55%가 수면에 대해 만족도를 보인 반면 한국인
은 41%만이 만족한다고 응답 
• 수면의 양과 질 모두 불만족스러운 상황이 수면 장애를 유발하면서 개인의 문제를 넘어 사
회경제적 손실로 확대될 수 있다는 지적이 제기됨 
- 수면 장애는 고혈압·심혈관 질환·당뇨병 등 만성 질환, 비만, 우울증 등을 유발하면서 전
반적인 삶의 만족도를 떨어뜨린다는 연구 결과가 이어짐 
- 보험연구원이 2023년 10월 30일 발표한 <수면 부족의 사회경제적 손실> 보고서에 따
르면, 수면 부족으로부터 발생하는 연간 경제적 손실은 국내총생산(GDP) 대비 0.85 
~2.92%로 추정  
[그림 1] OECD 회원국의 하루 평균 수면 시간  [그림 2] 양질의 수면을 방해하는 요소 
 
 
 
자료: KOTRA; OECD, 2021년 조사 재인용  자료: 필립스, 2021년 조사 
                                                           
1 한국, 호주, 브라질, 중국, 프랑스, 독일, 인도, 이탈리아, 일본, 네

In [62]:
!pip install pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 38.1 MB/s eta 0:00:00


In [63]:
from langchain_community.document_loaders import PDFMinerLoader

In [64]:
loader = PDFMinerLoader(FILE_PATH)

In [65]:
docs = loader.load()

In [66]:
len(docs)

1

In [67]:
print(docs[0].page_content)

2024. 4 

돈  되는  잠,  슬리포노믹스 

□  숙면하지  못하는  한국인과  슬리포노믹스의  등장 

□  다양한  분야로  확산되는  슬리포노믹스 

□  미래  먹거리로  부상하는  슬리포노믹스 

□  향후  전망과  시사점
 숙면하지  못하는  한국인과  슬리포노믹스(Sleeponomics)의  등장 

<  요      약  > 

•  한국인의  심각한  수면  시간  부족과  낮은  수면의  질은  개인의  문제를  넘어  사회경제적  손실로  확대될 

수  있다는  지적이  제기됨 

•  코로나19  이후  면역력을  키우고  건강  관리에  신경  쓰는  사람이  늘면서  수면에  대한  관심이  증가 

•  이는  수면(Sleep)과  경제학(Economics)의  합성어로,  각종  스트레스와  우울증으로  수면  부족에  시달리는 

현대인이  숙면에  도움이  되는  제품에  많은  돈을  투자하는  소비  현상인  ‘슬리포노믹스’  등장으로  이어짐 

 다양한  분야로  확산되는  슬리포노믹스 

•  인공지능(AI)과  사물인터넷(IoT)을  접목하여  수면  상태를  모니터링하는  앱이나  수면  건강에  도움을  주

는  기기를  개발하는  슬립테크(SleepTech) 

•  고급화를  통해  인플루언서들의  선택을  받으며  시장이  점차  확대되고  있는  프리미엄  침구류 

•  숙면에  도움을  주는  차와  천연  수면제  성분을  함유한  식음료 

•  셀프케어로  마음의  안정과  숙면을  유도하는  뷰티  제품 

 미래  먹거리로  부상하는  슬리포노믹스 

•  주요  빅테크  기업들은  슬리포노믹스를  차세대  먹거리로  선정하고  적극적인  공략에  나서며  시장  선점

을  위해  각축전을  펼칠  것으로  예상.  다양한  웨어러블  기기를  활용해  슬립테크  제품의  고도화를  추구

함으로써  슬리포노믹스  시장  선점을  시도 

•  기술력을  갖춘  중소기업들

In [69]:
from langchain_community.document_loaders import PDFMinerPDFasHTMLLoader

In [70]:
loader = PDFMinerPDFasHTMLLoader(FILE_PATH)

In [71]:
docs = loader.load()

In [72]:
len(docs)

1

In [74]:
print(docs[0].page_content[:300])

<html><head>
<meta http-equiv="Content-Type" content="text/html">
</head><body>
<span style="position:absolute; border: gray 1px solid; left:0px; top:50px; width:595px; height:841px;"></span>
<div style="position:absolute; top:50px;"><a name="1">Page 1</a></div>
<div style="position:absolute; border
